In [2]:
import openai
proxy = {
'http': 'http://localhost:7890',
'https': 'http://localhost:7890'
}

openai.proxy = proxy;

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1.0)

In [5]:
import asyncio
from langchain.document_loaders import PyMuPDFLoader
import os
from tqdm.notebook import tqdm

In [6]:
# !pip install -q chromadb

In [7]:
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

- 使用PyMuPDFLoader模块加载文件内容。
- 使用OpenAIEmbeddings模块为每个句子计算向量表示。
- 使用Chroma模块将文档转换为向量空间模型，并将其保存在磁盘上。
- 使用提供的ChatOpenAI对象提取文档中的贡献链。
- 将当前文件的结果添加到结果列表中。

In [8]:
import asyncio

In [9]:
# 定义一个函数，用于从给定文件路径中加载文件并提取相关信息
def load_file_and_extract(file_path):
    # 创建一个ChatOpenAI对象，model_name参数指定使用的模型名称，temperature参数指定生成文本时使用的温度值
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1.0)
    pwd = os.getcwd()
    # 获取文件路径下所有文件的文件名列表
    file_list = os.listdir(file_path)
    # 切换当前工作目录到文件路径下
    os.chdir(file_path)
    # 创建一个列表，用于保存每个文件的结果
    contri = []
    # 遍历文件列表，对每个文件进行处理
    for file in tqdm(file_list):
        # 加载PDF文件内容
        docs = PyMuPDFLoader(file).load()
        # 创建一个OpenAIEmbeddings对象，用于计算每个句子的向量表示
        embeddings = OpenAIEmbeddings()
        # 创建一个Chroma对象，用于将文档转换为向量空间模型，并存储在磁盘上
        vectorstore = Chroma.from_documents(docs, embeddings,collection_name=file.split('.')[0],persist_directory ='contribution')
        vectorstore.persist()
        # 使用提供的ChatOpenAI对象提取文档中的贡献链
        result = extract_contribution_chain(vectorstore,llm)
        # 将当前文件的结果添加到结果列表中
        contri.append(result)
    os.chdir(pwd)
    # 返回所有文件的结果
    return contri

这个函数的作用是从给定的向量空间模型中提取贡献链。具体来说，函数首先定义了一个查询语句，然后在向量空间模型中搜索与该查询语句相似的文档。接着，函数加载一个预定义的问题回答链，该链基于特定类型的贡献链，使用给定的预训练语言模型。最后，函数调用该问题回答链的run()方法，并指定输入文档和问题，以执行实际提取操作。最终，函数返回提取结果。

需要注意的是，该函数依赖于langchain.chains.question_answering模块中的load_qa_chain()函数。该函数将会加载一个已经训练好的基于预训练语言模型的问题回答链，进而对给定的文档和问题进行提取操作。

In [10]:
# 定义一个函数，用于从给定文件路径中加载文件并提取相关信息
def load_file_and_extract(file_path):
    # 创建一个ChatOpenAI对象，model_name参数指定使用的模型名称，temperature参数指定生成文本时使用的温度值
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1.0)
    pwd = os.getcwd()
    # 获取文件路径下所有文件的文件名列表
    file_list = os.listdir(file_path)
    # 切换当前工作目录到文件路径下
    os.chdir(file_path)
    # 创建一个列表，用于保存每个文件的结果
    contri = []
    # 遍历文件列表，对每个文件进行处理
    for file in tqdm(file_list):
        # 加载PDF文件内容
        docs = PyMuPDFLoader(file).load()
        # 创建一个OpenAIEmbeddings对象，用于计算每个句子的向量表示
        embeddings = OpenAIEmbeddings()
        # 创建一个Chroma对象，用于将文档转换为向量空间模型，并存储在磁盘上
        vectorstore = Chroma.from_documents(docs, embeddings,collection_name=file.split('.')[0],persist_directory ='contribution')
        vectorstore.persist()
        # 使用提供的ChatOpenAI对象提取文档中的贡献链
        result = extract_contribution_chain_old(vectorstore,llm)
        # 将当前文件的结果添加到结果列表中
        contri.append(result)
    os.chdir(pwd)
    # 返回所有文件的结果
    return contri

In [11]:
# 导入函数load_qa_chain，用于加载问题回答链
from langchain.chains.question_answering import load_qa_chain

In [12]:
# 定义一个函数，用于从给定向量空间模型中提取贡献链
def extract_contribution_chain_old(vectorstore,llm):
    # 定义一个查询语句
    query = "What are the main findings of this article, and what were its key points of innovation? must use less than 100 words!"
    
    # 在向量空间模型中搜索与查询语句相似的文档
    doc =  vectorstore.similarity_search(query, 2, include_metadata=True)
    
    # 加载问题回答链，llm参数指定使用的预训练语言模型，chain_type参数指定使用的贡献链类型
    chain = load_qa_chain(llm, chain_type="stuff")
    
    # 执行提取操作，input_documents参数指定输入的文档，question参数指定要问的问题
    resp = chain.run(input_documents=doc, question=query)
    
    # 返回提取结果
    return resp

In [13]:
async def async_load_file_and_extract(file_path):
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1.0)
    pwd = os.getcwd()
    file_list = os.listdir(file_path)
    os.chdir(file_path)
    contri = []
    tasks = []
    for file in file_list:
        docs = PyMuPDFLoader(file).load()
        embeddings = OpenAIEmbeddings()
        vectorstore = Chroma.from_documents(docs, embeddings,collection_name=file.split('.')[0],persist_directory ='contribution')
        vectorstore.persist()
        tasks.append(extract_contribution_chain(vectorstore,llm))
    os.chdir(pwd)
    results = await asyncio.gather(*tasks)
    return results

async def extract_contribution_chain(vectorstore,llm):
    query = "What are the main findings of this article, and what were its key points of innovation? must use less than 100 words!"
    doc =  vectorstore.similarity_search(query, 2, include_metadata=True)
    chain = load_qa_chain(llm, chain_type="stuff")
    resp = await chain.arun(input_documents=doc, question=query)
    return resp

In [14]:
os.getcwd()

'F:\\notebook\\langchain-proj'

In [36]:
!rmdir /s /q  "F:\\notebook\\langchain-proj\\pdf̧\\contribution"
!rmdir /s /q  "F:\\notebook\\langchain-proj\\pdf̧\\.chroma"

系统找不到指定的路径。
系统找不到指定的路径。


In [15]:
import time

同步模式

In [17]:
s = time.perf_counter()
# file_path = "./pdf"
file_path = 'F:\\notebook\\langchain-proj\\pdf'
resp = load_file_and_extract(file_path)
elapsed = time.perf_counter() - s

  0%|          | 0/10 [00:00<?, ?it/s]

Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution


In [18]:
elapsed

173.46237700000003

异步模式

In [17]:
s = time.perf_counter()
# If running this outside of Jupyter, use asyncio.run(generate_concurrently())
# await generate_concurrently()

# file_path = "./pdf" 
file_path = 'F:\\notebook\\langchain-proj\\pdf'

# 如果在Jupyter之外运行,使用asyncio.run启动事件循环
# asyncio.run(main())

resp = await async_load_file_and_extract(file_path)
# 使用resp获取结果
elapsed = time.perf_counter() - s

Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution
Using embedded DuckDB with persistence: data will be stored in: contribution


In [18]:
resp

['This article examines how China\'s "rule of mandates" governing system, characterized by authoritarian mandates from above combined with local level discretion, hinders efforts to detect corruption but also how it might be used to channel corruption into certain areas. The paper offers possible explanations for why corruption has not hindered economic development, as well as prescribing reforms to combat corruption that should "engage the mandates system, not simply the legal system." It concludes by offering potential avenues for further research into the impact of the rule of mandates on corruption.',
 'The article examines the impact of China\'s political congresses on local officials\' incentives to pursue short-term economic growth at the expense of environmental protection. Using industrial sulfur dioxide emissions as a measure of pollution, the authors find that the convening of political congresses is associated with a "political pollution cycle," where officials prioritize e

In [19]:
elapsed

53.51185429999987

In [20]:
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [21]:
def search_for_similarity(my_con, lit_con,k):

    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_texts(lit_con, embeddings)
    
    doc =  vectorstore.similarity_search(my_con, k)
    
    doc = "\n\n\n".join([element.page_content for element in doc])
    
    return doc

In [22]:
my_con ="""

This paper tests whether local officials will engage in opportunistic behavior before the convening of the Provincial Communist Party Congress (PCPC), reducing the carbon emissions before this major political event. It aims at illustrating whether there is a political carbon emission cycle in China, showing lower carbon emissions before the PCPC and high carbon emissions after the PCPC. This is an important question, because the opportunistic behaviors in reducing carbon emissions will impede the long-term technological and structural adjustments in the CO2-emitting industry, so that the ‘Peak Carbon Dioxide Emission’ and ‘Carbon Neutral’ may not be realized in the future. We first calculate the carbon emissions in 283 prefecture-level cities from 2006 to 2019 in China through summing carbon emissions generated from direct energy consumption and indirect energy consumption. Then, we collect the convening date of the PCPC in each province and the local officials’ data, including the tenure, age, education and the political turnover year of the mayor and party secretaries in prefecture-level cities. Due to the strong exogeneity of the political cycle, we use Fixed-Effect model to conduct the empirical research. Our empirical analysis shows that the political carbon emissions cycle indeed exists in China, showing a decrease in carbon emissions before the PCPC and rise again after the PCPC. We also find the political carbon emission cycle is not belong to environmental political business cycle. Moreover, political business cycle and political carbon emission cycle co-exist in China at present, indicating carbon emissions reduction have already been a high priority target which is equal to the economic growth in cadre evaluation system. Finally, we find the local officials’ age have heterogeneous effect on the political carbon emission cycle and the political carbon emission cycle only emerge after 2010. These two results are strong evidence supporting the political carbon emission cycle is a result of local official’s opportunistic behavior rather than a natural fluctuation.

"""

In [23]:
con_list = search_for_similarity(my_con, resp,5)

Using embedded DuckDB without persistence: data will be transient


In [24]:
con_list

'The article examines the impact of China\'s political congresses on local officials\' incentives to pursue short-term economic growth at the expense of environmental protection. Using industrial sulfur dioxide emissions as a measure of pollution, the authors find that the convening of political congresses is associated with a "political pollution cycle," where officials prioritize economic growth over environmental protection. The authors also identify a shift in officials\' incentives after the implementation of environmental regulations and emphasize the importance of considering political cycles in studying the effectiveness of environmental policy.\n\n\nThe article analyzes the political-business cycle in China, focusing on its impact on environmental regulations. The study uses annual PM2.5 data and finds evidence suggesting that the years leading to a prefecture party secretary\'s turnover are associated with relaxed environmental regulations and higher pollution levels. The stu

In [25]:
question_template = """

You are a helpful writing assistant in writing academic paper. 

Given the main finding of my paper and main finding of others' paper, you need to write the innovation part of my paper.

Don't only repeat the finding of my paper when writing innovation part of my paper.

You need to pay attention to the following three points:

1.Discuss why the findings or research methods of my article are different from others', and why this difference is highly significant and innovative.

2.Discuss why our research results are highly meaningful and the contributions they will make to the current academic community.

3.Maintain clarity and logical coherence throughout the discussion, selectively emphasizing the significance of the research findings in the description, but exercising caution in the choice of words.

===========
The following are findings of my paper:

{mine}
===========

The following are findings of others' paper:

{others}


"""

from langchain.chains.question_answering import load_qa_chain
from langchain import LLMChain
from langchain.prompts import PromptTemplate

PROMPT = PromptTemplate(template = question_template,
                        input_variables=['mine','others'])

llm_chain = LLMChain(llm=llm,prompt = PROMPT)

In [26]:
llm_chain.run({"mine":my_con,"others":con_list})

"Innovation:\n\nOur study makes significant contributions to the existing literature on the relationship between politics and the environment in China. Unlike previous studies that focused on the impact of political congresses or the political-business cycle on local pollution, our study investigates whether local officials engage in opportunistic behavior to reduce carbon emissions before the convening of the Provincial Communist Party Congress (PCPC), and whether this behavior hinders long-term technological and structural adjustments in the CO2-emitting industry. \n\nOur study reveals the existence of a political carbon emissions cycle which shows lower carbon emissions before the PCPC and high carbon emissions after the PCPC. This finding is highly significant and innovative as it sheds light on the role of local officials in opportunistic behavior, challenging the conventional wisdom that the political-business cycle is the only driving force in local pollution. This finding is pa

In [27]:
diff_template = """

You are a helpful writing assistant in writing academic paper. 

Given the main finding of my paper and main finding of others' paper, compare my research findings and research methods with those of other articles one by one.

Please write a discussion in detail on how my research methods and findings differ from those of other articles, and specifically, in what aspects these differences exist,and why these difference matters?

Ensuring the word count is not less than 1000.

===========
The following are my paper:

{mine}
===========

The following are others' article:

{others}


"""

from langchain.chains.question_answering import load_qa_chain
from langchain import LLMChain
from langchain.prompts import PromptTemplate

PROMPT = PromptTemplate(template = diff_template,
                        input_variables=['mine','others'])

llm_chain = LLMChain(llm=llm,prompt = PROMPT)

In [28]:
llm_chain.run({"mine":my_con,"others":con_list})

"In comparison to the aforementioned articles, our study mainly focuses on the political carbon emission cycle in China. While two of the articles examined political pollution cycles and political business cycles, they did not specifically explore carbon emissions. Our study aimed at determining whether local officials in China engage in opportunistic behavior before the convening of the Provincial Communist Party Congress, reducing carbon emissions before this major political event. Moreover, our paper also aimed at illustrating whether carbon emissions exhibit a cycle preceding the PCPC, which could have detrimental effects on long-term technological and structural adjustments in CO2-emitting industries.\n\nIn terms of methodology, our study collected data on carbon emissions in 283 prefecture-level cities in China from 2006 to 2019 through summing carbon emissions generated from direct energy consumption and indirect energy consumption. We then collected the convening date of the PC